In [1]:
# = Import things
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets

In [12]:
#Model
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,10),
        ) #Here we choose 4 layers with 512, 256, 128 and output as 10 neurons

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
# Prepare the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Grayscale(), #set to grayscale
                                transforms.Normalize((0.5,), (0.5,))])

#train_dataset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
#train_data = DataLoader(train_dataset, batch_size=1024, shuffle=True) #MNIST datasets from torchvision

#or by using own datasets
train_dataset = datasets.ImageFolder("C:\\Users\\username\\..." transform=transform) #Change the file path
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)

# Model definition
model = MyModel()
# Check if CUDA is available, else use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Move the model to the device
model.to(device)

# Loss function definition :
loss_func = nn.CrossEntropyLoss()
# Optimizer definition
optimizer = torch.optim.Adam(model.parameters(), lr=0.002) #Turns out Adam optimizer have better result that SGD

# Train the model
for epoch in range(15):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_data, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  #set to 0 gradiant
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward() #Backward propagation
        optimizer.step()

        running_loss += loss.item() 

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_data)}")
print("Training done")

In [ ]:
testset = datasets.ImageFolder("C:\\Users\\username\\...",transform = transform)#Change the file path
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
# or by using MNIST datasets
#testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
#testloader = DataLoader(testset, batch_size=64, shuffle=True)

def test_loop(test_dataloader, model, loss_func):
    model.eval()
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    with torch.no_grad():
        for Im, label in test_dataloader:
            Im, label = Im.to(device), label.to(device)
            pred = model(Im)
            test_loss += loss_func(pred, label).item()
            correct += (pred.argmax(1) == label).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

test_loop(testloader, model, loss_func)


In [ ]:
# Import additional required libraries
import matplotlib.pyplot as plt
import numpy as np


# Load test data
testset = datasets.ImageFolder("C:\\Users\\username\\...",transform = transform)#Change the file path
testloader = DataLoader(testset, batch_size=64, shuffle=True)


# Test the model and visualize predictions
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get a batch of test data
dataiter = iter(testloader)
images, labels = next(dataiter)

model.to(torch.device('cpu')) # moving model to cpu

# Predict labels
model.eval() #set to evaluation mode
with torch.no_grad(): #dissable gradients (takes memory)
    outputs = model(images)
_, predicted = torch.max(outputs, 1)


# Show number and prediction side by side
fig = plt.figure(figsize=(25, 4)) # create a figure with a larger size
for idx in range(64): # loop over all images in the batch
    ax = fig.add_subplot(4, 16, idx+1, xticks=[], yticks=[]) # create a subplot for each image
    ax.imshow(images[idx].squeeze(), cmap='gray') # show the image in grayscale
    ax.text(0.5, -0.5, f'{labels[idx].item()} ({predicted[idx].item()})', ha='center', color='green' if labels[idx]==predicted[idx] else 'red') # add text annotation with the number and prediction, using green color for correct predictions and red color for wrong predictions
plt.show() # show the figure

In [27]:
#Save to .pth
torch.save(model.state_dict(), "C:\\Users\\username\\...") #Change the file path